# Imports and configs

In [1]:
!pip install -q autogluon.tabular ray==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 101.4 MB/s eta 0:00:00


In [2]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np
import shutil

In [3]:
class CFG:
    train_path = '/kaggle/input/ai-competition-africa/train.csv'
    test_path = '/kaggle/input/ai-competition-africa/test.csv'
    sample_sub_path = '/kaggle/input/ai-competition-africa/sample_submission.csv'
    
    target = 'quality'
    time_limit = 3600 * 11

# Loading data

In [4]:
train = pd.read_csv(CFG.train_path, index_col='id')
test = pd.read_csv(CFG.test_path, index_col='id')

train[CFG.target] = train[CFG.target].astype(int) - 3

# Fitting the predictor

In [5]:
predictor = TabularPredictor(
    problem_type='multiclass',
    eval_metric='quadratic_kappa',
    label=CFG.target,
    verbosity=2
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241223_122212"


In [6]:
predictor.fit(
    train_data=train,
    time_limit=CFG.time_limit,
    excluded_model_types = ["KNN"],
    presets='experimental_quality',
    ag_args_fit={
        'num_gpus': 2, 
        'num_cpus': 4
    }
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.20 GB / 31.35 GB (96.3%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['experimental_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect 

In [7]:
predictor.leaderboard(silent=True).style.background_gradient(subset=['score_val'], cmap='RdYlGn')

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.386294,quadratic_kappa,1.607337,10.404621,0.003816,0.860987,2,True,19
1,RandomForest_r34_BAG_L1,0.382710,quadratic_kappa,0.413494,3.492249,0.413494,3.492249,1,True,11
2,ExtraTrees_r4_BAG_L1,0.380357,quadratic_kappa,0.425124,1.728958,0.425124,1.728958,1,True,12
3,ExtraTrees_r172_BAG_L1,0.379608,quadratic_kappa,0.536558,2.993032,0.536558,2.993032,1,True,7
4,ExtraTrees_r178_BAG_L1,0.374656,quadratic_kappa,0.516527,2.448700,0.516527,2.448700,1,True,13
5,RandomForest_r127_BAG_L1,0.371249,quadratic_kappa,0.499109,9.262070,0.499109,9.262070,1,True,10
6,RandomForest_r15_BAG_L1,0.362984,quadratic_kappa,0.533401,7.386339,0.533401,7.386339,1,True,15
7,ExtraTrees_r42_BAG_L1,0.360967,quadratic_kappa,0.653469,3.058353,0.653469,3.058353,1,True,5
8,RandomForest_r39_BAG_L1,0.360663,quadratic_kappa,0.577676,7.817631,0.577676,7.817631,1,True,8
9,RandomForestEntr_BAG_L1,0.360189,quadratic_kappa,0.607576,5.471784,0.607576,5.471784,1,True,2


# Creating a submission file

In [8]:
test_pred_probs = predictor.predict_proba(test)
sub = pd.read_csv(CFG.sample_sub_path)
sub[CFG.target] = np.argmax(test_pred_probs, axis=1)
sub[CFG.target] = sub[CFG.target] + 3
sub.to_csv('sub_autogluon.csv', index=False)
sub.head()

,id,quality
0,15000,5
1,15001,5
2,15002,6
3,15003,6
4,15004,5


In [9]:
shutil.rmtree("AutogluonModels")